In [10]:
# Load python libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys
sys.path.append('C:/Users/PC/AppData/Local/Programs/Python/Python36/Lib/site-packages')

%matplotlib inline
plt.style.use('seaborn')

train_path = 'data/train.csv'

In [3]:
%%time
# As train data is huge (5gb), we need to find out how we can optimise convertion to dataframe
with open(train_path) as file:
    n_rows = len(file.readlines())

print (f'Number of rows: {n_rows}')

Number of rows: 55423857
Wall time: 28 s


In [11]:
# Read in part of data for a peek
df_temp = pd.read_csv(train_path, nrows=10000)
df_temp.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [12]:
df_temp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
key                  10000 non-null object
fare_amount          10000 non-null float64
pickup_datetime      10000 non-null object
pickup_longitude     10000 non-null float64
pickup_latitude      10000 non-null float64
dropoff_longitude    10000 non-null float64
dropoff_latitude     10000 non-null float64
passenger_count      10000 non-null int64
dtypes: float64(5), int64(1), object(2)
memory usage: 2.0 MB


* Train data consists of around 55 million rows, some optimisation will be needed.
* There are 2 columns with object dtype which takes up the most memory
    * 'key' column can be omitted as it is a repeat of 'pickup_datetime' column
    * 'pickup_datetime' column should be converted to datetime dtype to save memory
* float32 (7 decimal places) should be enough for the longitude and latitude values. 
* uint8 is suitable for passenger_count as it cannot be negative and is not more than 255.

In [15]:
# Allocating the suitable dtypes for optimal memory usage
dtypes = {'fare_amount': 'float32',
          'pickup_datetime': 'str', 
          'pickup_longitude': 'float32',
          'pickup_latitude': 'float32',
          'dropoff_longitude': 'float32',
          'dropoff_latitude': 'float32',
          'passenger_count': 'uint8'}

# Selecting relevant columns
cols = list(dtypes.keys())

# 10 million rows per chunk
chunksize = 10_000_000 

In [16]:
df_list = [] # list to hold the batch dataframe

for df_chunk in pd.read_csv(train_path, usecols=cols, dtype=dtypes, chunksize=chunksize):
    
    df_chunk['pickup_datetime'] = df_chunk['pickup_datetime'].str.slice(0, 16)
    df_chunk['pickup_datetime'] = pd.to_datetime(df_chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    df_list.append(df_chunk) 

# Merge all dataframes into one dataframe
train_df = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
fare_amount          float32
pickup_datetime      datetime64[ns, UTC]
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.5 GB


In [25]:
# Convert df into feather format 
train_df.to_feather('nyc_taxi_train.feather')

# Loading feather df
train_df_new = pd.read_feather('nyc_taxi_train.feather')
train_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
fare_amount          float32
pickup_datetime      datetime64[ns, UTC]
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.5 GB


In [27]:
train_df_new.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542386e+07,5.542386e+07,5.542386e+07,5.542348e+07,5.542348e+07,5.542386e+07
mean,8.077921e+00,-3.875290e+01,1.937328e+01,-3.875261e+01,1.937341e+01,1.685380e+00
std,2.055127e+01,2.556202e+01,1.414234e+01,2.553839e+01,1.412191e+01,1.327664e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396368e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


Some observations:
* Minimum of fare_amount is negative
* Some missing values for dropoff_longitude and dropoff_latitude
* Max fare_amount seem unrealistic

In [55]:
neg_fare = len(train_df_new[train_df_new['fare_amount'] < 0])
print(f'No. of negative fare_amount: {neg_fare}')
pct = neg_fare / 55423856 * 100
print(f'Percentage of negative fare_amount: {pct:.4f}%')

No. of negative fare_amount: 0
Percentage of negative fare_amount: 0.0000%


In [51]:
train_df_new = train_df_new[train_df_new['fare_amount'] >= 0]

In [52]:
train_df_new.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542140e+07,5.542140e+07,5.542140e+07,5.542103e+07,5.542103e+07,5.542140e+07
mean,8.078549e+00,-3.875462e+01,1.937414e+01,-3.875433e+01,1.937427e+01,1.685369e+00
std,2.055111e+01,2.556259e+01,1.414266e+01,2.553895e+01,1.412200e+01,1.327656e+00
min,0.000000e+00,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396368e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


In [53]:
train_df_new.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55421402 entries, 0 to 55423855
Data columns (total 7 columns):
fare_amount          float32
pickup_datetime      datetime64[ns, UTC]
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.9 GB
